In [1]:
#前期的工作在于构建3个嵌入矩阵

In [2]:
#使用pytorch构建深度学习模型有4步
#1.定义网络的各个层
#2.定义各个对层的操作
#3.定义损失函数
#4.更新网络参数

In [3]:
import argparse
import torch
import time
import json
import numpy as np
import math
import random
import codecs
import os
from utils_train import load_data, generate_repositioning, generate_repositioning_data, predict_aspects_nums
from utils_train import generate_start_index, train_as, merge_data, merge_data_final, save_batch, clean_batch
from utils_train import return_predicted_nums
from generate_train_data import generate_aspects_nums_train_data, merge_aspects_nums_data, merge_aspects_nums_final_data
from process_batch_data import generate_aspects_nums_batch_data
from json_to_csv import json_to_csv

/home/xinke901/anaconda3/envs/de-cnn/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/xinke901/anaconda3/envs/de-cnn/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/xinke901/anaconda3/envs/de-cnn/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/xinke901/anaconda3/envs/de-cnn/lib

In [4]:
np.random.seed(1337)
random.seed(1337)
torch.manual_seed(1337)
torch.cuda.manual_seed(1337)

In [5]:
"""
输入：训练集或者验证集的数据和标签，batch_size，
输出：
代码思路：
"""
def batch_generator(X, y, batch_size=128, return_idx=False, crf=True):
    #以batch_size为步长
    for offset in range(0, X.shape[0], batch_size):
        batch_X_len=np.sum(X[offset:offset+batch_size]!=0, axis=1)
        batch_idx=batch_X_len.argsort()[::-1]
        batch_X_len=batch_X_len[batch_idx]
        batch_X_mask=(X[offset:offset+batch_size]!=0)[batch_idx].astype(np.uint8)
        batch_X=X[offset:offset+batch_size][batch_idx]
        batch_y=y[offset:offset+batch_size][batch_idx]
        batch_X = torch.autograd.Variable(torch.from_numpy(batch_X).long().cuda())
        batch_X_mask=torch.autograd.Variable(torch.from_numpy(batch_X_mask).long().cuda())
        batch_y = torch.autograd.Variable(torch.from_numpy(batch_y).long().cuda())
        if len(batch_y.size())==2 and not crf:
            batch_y=torch.nn.utils.rnn.pack_padded_sequence(batch_y, batch_X_len, batch_first=True)
        if return_idx: #in testing, need to sort back.
            yield (batch_X, batch_y, batch_X_len, batch_X_mask, batch_idx)
        else:
            yield (batch_X, batch_y, batch_X_len, batch_X_mask)

In [6]:
"""
输入：通用领域嵌入，特定领域嵌入，3个参数，分类个数、dropout率、是否使用CRF
输出：3个标签的概率
代码思路：
"""
class Model(torch.nn.Module):
    #定义神经网络层
    def __init__(self, gen_emb, domain_emb, num_classes=3, dropout=0.55, crf=True):
        super(Model, self).__init__()
        #构建通用领域嵌入矩阵，先生成相应大小的空间，再将参数传进去
        self.gen_embedding = torch.nn.Embedding(gen_emb.shape[0], gen_emb.shape[1])
        self.gen_embedding.weight=torch.nn.Parameter(torch.from_numpy(gen_emb), requires_grad=False)
        #构建特定领域嵌入矩阵，先生成相应大小的空间，再将参数传进去
        
        #设定模型的卷积层参数
        #第一层第一种卷积核,输入维度是400，输出维度是128，跨步是5，填充是2
        self.conv1=torch.nn.Conv1d(gen_emb.shape[1], 128, 5, padding=2)
        self.conv2=torch.nn.Conv1d(gen_emb.shape[1], 128, 3, padding=1)
        self.dropout=torch.nn.Dropout(dropout)
        
        #剩下三层的参数相同，输入维度是256，输出维度是128，跨步是5，填充是2
        self.conv3=torch.nn.Conv1d(256, 256, 5, padding=2)
        self.conv4=torch.nn.Conv1d(256, 256, 5, padding=2)
        self.conv5=torch.nn.Conv1d(256, 256, 5, padding=2)
        #设置解码层1：就是一个全连接层进行分类，分为3类
        self.linear_ae=torch.nn.Linear(256, num_classes)
        self.crf_flag=crf
        #设置解码层2：CRF
        if self.crf_flag:
            from allennlp.modules import ConditionalRandomField
            self.crf=ConditionalRandomField(num_classes)
        
    #定义操作
    def forward(self, x, x_len, x_mask, x_tag=None, testing=False):
        #cat是拼接的意思，0是竖着拼，1是横着拼，2是（3维的情况）
        #这段代码好像是传入batchsize的单词的编号，然后去除这些单词嵌入
        x_emb=self.gen_embedding(x)
        x_emb=self.dropout(x_emb).transpose(1, 2)
        x_conv=torch.nn.functional.relu(torch.cat((self.conv1(x_emb), self.conv2(x_emb)), dim=1))
        
        #3个相同的卷积层
        x_conv=self.dropout(x_conv)
        x_conv=torch.nn.functional.relu(self.conv3(x_conv))
        x_conv=self.dropout(x_conv)
        x_conv=torch.nn.functional.relu(self.conv4(x_conv))
        x_conv=self.dropout(x_conv)
        x_conv=torch.nn.functional.relu(self.conv5(x_conv))
        x_conv=x_conv.transpose(1, 2)
        x_logit=self.linear_ae(x_conv)
        if testing:
            if self.crf_flag:
                score=self.crf.viterbi_tags(x_logit, x_mask)
            else:
                x_logit=x_logit.transpose(2, 0)
                score=torch.nn.functional.log_softmax(x_logit).transpose(2, 0)
        else:
            if self.crf_flag:
                score=-self.crf(x_logit, x_tag, x_mask)
            else:
                x_logit=torch.nn.utils.rnn.pack_padded_sequence(x_logit, x_len, batch_first=True)
                score=torch.nn.functional.nll_loss(torch.nn.functional.log_softmax(x_logit.data), x_tag.data)
        return score

In [7]:
def valid_loss(model, valid_X, valid_y, crf=True):
    model.eval()
    losses=[]
    for batch in batch_generator(valid_X, valid_y, crf=crf):
        batch_valid_X, batch_valid_y, batch_valid_X_len, batch_valid_X_mask=batch
        loss=model(batch_valid_X, batch_valid_X_len, batch_valid_X_mask, batch_valid_y)
        losses.append(loss.item())
    model.train()
    return sum(losses)/len(losses)

In [8]:
word_idx_fn = 'data/prep_data/word_idx.json'
with open(word_idx_fn) as f:
    word_idx=json.load(f)
idx_word={}
for key,val in word_idx.items():
    idx_word[val]=key

In [9]:
def train(train_X, train_y, valid_X, valid_y, model, model_fn, optimizer, parameters, run_epoch, epochs, batch_size, crf, generate_data, add_num_loss):
    best_loss=float("inf")
    valid_history=[]
    train_history=[]
    for epoch in range(epochs):
        pred_y=np.zeros((train_X.shape[0], train_X.shape[1]), np.int16)
        offset = range(0, train_X.shape[0], batch_size)
        i_th = 0
        results = []
        for batch in batch_generator(train_X, train_y, batch_size, crf=crf):
            batch_train_X, batch_train_y, batch_train_X_len, batch_train_X_mask=batch
            
            #最开始的时候loss变化很大，等稳定了再加入，而且太慢了。。。。。。
            if add_num_loss and epoch >= 250:
                results_num = []
                #先按照之前的格式输出每个batch，然后识别出预测的方面，再组成csv格式
                model.eval()
                batch_pred_y=model(batch_train_X, batch_train_X_len, batch_train_X_mask, testing=True)
                model.train()
                
                for j_th in range(len(batch_train_X)):
                    result = []
                    #需要把tensor转成np
                    words_num = [i for i in batch_train_X[j_th].cpu().numpy()]
                    words_str = []
                    #把数字转换成单词
                    for w in words_num:
                        if(w != 0):
                            words_str.append(idx_word[w])
                    #需要把tensor转成np
                    gold = [i for i in batch_train_y[j_th].cpu().numpy()]
                    #预测的原始原始输出需要处理
                    pred = batch_pred_y[j_th][0]
                    for words_str, gold, pred in zip(words_str, gold, pred):
                        result.append(" ".join([words_str, str(gold), str(pred)]))
                    results_num.append(result)
                save_batch(results_num)
                #从原始输出中找出提取的方面
                pred_aspects_nums = generate_aspects_nums_batch_data()
                total_pred_aspects_nums = 0
                #print('len of pred_aspects_nums', len(pred_aspects_nums))
                #print('pred_aspects_nums', pred_aspects_nums)
                
                for i in pred_aspects_nums:
                    total_pred_aspects_nums += i
                if total_pred_aspects_nums != 0:
                    json_to_csv()
                    #调用bert进行预测，返回预测的每个句子的结果
                    predict_aspects_nums()
                    predicted_nums = return_predicted_nums()
                    num_loss = 0
                    not_zero_pred_aspects_nums = []
                    for i in range(len(pred_aspects_nums)):
                        if pred_aspects_nums[i] != 0:
                            not_zero_pred_aspects_nums.append(pred_aspects_nums[i])
                    """
                    print('len of not_zero_pred_aspects_nums', len(not_zero_pred_aspects_nums))
                    print('not_zero_pred_aspects_nums', not_zero_pred_aspects_nums)
                    print('len of predicted_nums', len(predicted_nums))
                    print('predicted_nums', predicted_nums)
                    if len(not_zero_pred_aspects_nums) != len(predicted_nums):
                        print('len of not_zero_pred_aspects_nums', len(not_zero_pred_aspects_nums))
                        print('not_zero_pred_aspects_nums', not_zero_pred_aspects_nums)
                        print('len of predicted_nums', len(predicted_nums))
                        print('predicted_nums', predicted_nums)
                        assert len(not_zero_pred_aspects_nums) == len(predicted_nums)
                    """
                    #存在一个BUG--predicted_nums和not_zero_pred_aspects_nums长度可能不等
                    for i in range(len(predicted_nums)):
                        num_loss += (predicted_nums[i] - not_zero_pred_aspects_nums[i]) ** 2
                    num_loss = (num_loss / len(predicted_nums)) * 10
                    print('bert计算的loss是：', num_loss)
                else:
                    #因为所有的句子长度为83
                    num_loss = 83
                #生成bert需要的输入格式
                #clean_batch('output_data/batch_result.utf8')
                #clean_batch('output_data/batch_result.json')
                #clean_batch('output_data/batch_result.csv')
                
            #开始训练
            #这里的model是Model里面的forward函数
            if add_num_loss and epoch >= 250:
                loss=model(batch_train_X, batch_train_X_len, batch_train_X_mask, batch_train_y) + num_loss
                print('loss是：', loss)
            else:
                loss=model(batch_train_X, batch_train_X_len, batch_train_X_mask, batch_train_y)
            #print('loss是：', loss)
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm(parameters, 1.)
            optimizer.step()
            
            if generate_data:
                """
                输出预测标签
                作用：输出每行句子的每个单词的预测标签
                """
                model.eval()
                batch_print_X_len=np.sum(train_X[offset[i_th]:offset[i_th]+batch_size]!=0, axis=1)
                batch_idx=batch_print_X_len.argsort()[::-1]
                batch_print_X_len=batch_print_X_len[batch_idx]
                batch_print_X_mask=(train_X[offset[i_th]:offset[i_th]+batch_size]!=0)[batch_idx].astype(np.uint8)
                batch_print_X=train_X[offset[i_th]:offset[i_th]+batch_size][batch_idx]
                batch_print_X_mask=torch.autograd.Variable(torch.from_numpy(batch_print_X_mask).long().cuda())
                batch_print_X=torch.autograd.Variable(torch.from_numpy(batch_print_X).long().cuda())
                batch_pred_y=model(batch_print_X, batch_print_X_len, batch_print_X_mask, testing=True)
                r_idx=batch_idx.argsort()
                if crf:
                    batch_pred_y=[batch_pred_y[idx] for idx in r_idx]
                    for ix in range(len(batch_pred_y)):
                        for jx in range(len(batch_pred_y[ix][0])):
                            pred_y[offset[i_th]+ix,jx]=batch_pred_y[ix][0][jx]
                else:
                    batch_pred_y=batch_pred_y.data.cpu().numpy().argmax(axis=2)[r_idx]
                    pred_y[offset[i_th]:offset[i_th]+batch_size,:batch_pred_y.shape[1]]=batch_pred_y
                model.train()
                i_th += 1
            
        if generate_data:
            #写入句子、真实标签、预测标签
            #要求：一行句子一行句子地读，然后一个单词一个单词地写，每个句子读完后以回车标志结束
            for j_th in range(len(train_X)):
                result = []
                words_num = train_X[j_th]
                words_str = []
                #把数字转换成单词
                for w in words_num:
                    if(w != 0):
                        words_str.append(idx_word[w])
                gold = train_y[j_th]
                pred = pred_y[j_th]
                for words_str, gold, pred in zip(words_str, gold, pred):
                    result.append(" ".join([words_str, str(gold), str(pred)]))
                results.append(result)
            train_as(results, epoch, run_epoch)

            context, query, answer, answer_start_index = load_data(epoch, run_epoch)
            update_index = generate_start_index(context, answer_start_index, answer)
            generate_repositioning_data(context, query, answer, update_index, epoch, run_epoch)
            generate_aspects_nums_train_data(run_epoch, epoch)
            """
            输出预测标签结束
            """
        
        #一轮训练完了，计算loss
        loss=valid_loss(model, train_X, train_y, crf=crf)
        train_history.append(loss)
        loss=valid_loss(model, valid_X, valid_y, crf=crf)
        valid_history.append(loss)
        if loss<best_loss:
            best_loss=loss
            torch.save(model, model_fn)
        shuffle_idx=np.random.permutation(len(train_X))
        train_X=train_X[shuffle_idx]
        train_y=train_y[shuffle_idx]
        if(epoch % 10 == 0):
            print(str(epoch) + '/' + str(epochs))
    model=torch.load(model_fn)
    return train_history, valid_history

In [10]:
def run(domain, data_dir, model_dir, valid_split, runs, epochs, lr, dropout, batch_size, crf, generate_data, add_num_loss):
    #2个单词嵌入
    gen_emb=np.load(data_dir+"gen.vec.npy")
    domain_emb=np.load(data_dir+domain+"_emb.vec.npy")
    #训练集
    ae_data=np.load(data_dir+domain+".npz")
    
    #分割训练集，此处是固定分割
    valid_X=ae_data['train_X'][-valid_split:]
    valid_y=ae_data['train_y'][-valid_split:]
    train_X=ae_data['train_X'][:-valid_split]
    train_y=ae_data['train_y'][:-valid_split]

    print("数据集总大小：", len(ae_data['train_X']))
    print("训练集大小：", len(train_X))
    print("验证集大小：", len(valid_X))

    if add_num_loss:
        model_dir = 'model_num/'
    
    for r in range(runs):
        print('正在训练第 ' + str(r + 1) + '轮')
        #初始化model
        model=Model(gen_emb, domain_emb, 3, dropout, crf)
        model.cuda()
        print(model)
        parameters = [p for p in model.parameters() if p.requires_grad]
        optimizer=torch.optim.Adam(parameters, lr=lr)
        #调用train模块
        train_history, valid_history=train(train_X, train_y, valid_X, valid_y, model, model_dir+domain+str(r), 
                                           optimizer, parameters, r, epochs, batch_size, crf, generate_data, add_num_loss)
        if generate_data:
            merge_data(r, epochs)
            merge_aspects_nums_data(r, epochs)
    if generate_data:
        merge_data_final(runs, epochs)
        merge_aspects_nums_final_data(runs, epochs)

In [11]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument('--model_dir', type=str, default="model/")
    parser.add_argument('--batch_size', type=int, default=128)
    parser.add_argument('--epochs', type=int, default=300)
    parser.add_argument('--runs', type=int, default=5)
    parser.add_argument('--domain', type=str, default="laptop")
    parser.add_argument('--data_dir', type=str, default="data/prep_data/")
    parser.add_argument('--valid', type=int, default=150) #number of validation data.
    parser.add_argument('--lr', type=float, default=0.0001)
    parser.add_argument('--dropout', type=float, default=0.55)
    parser.add_argument('--crf', type=bool, default=True)
    parser.add_argument('--generate_data', type=bool, default=False)
    parser.add_argument('--add_num_loss', type=bool, default=True)
    args = parser.parse_known_args()[0]

    run(args.domain, args.data_dir, args.model_dir, args.valid, args.runs, args.epochs, args.lr, args.dropout, args.batch_size, args.crf, args.generate_data, args.add_num_loss)

数据集总大小： 3045
训练集大小： 2895
验证集大小： 150
正在训练第 1轮
Model(
  (gen_embedding): Embedding(8518, 300)
  (conv1): Conv1d(300, 128, kernel_size=(5,), stride=(1,), padding=(2,))
  (conv2): Conv1d(300, 128, kernel_size=(3,), stride=(1,), padding=(1,))
  (dropout): Dropout(p=0.55, inplace=False)
  (conv3): Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
  (conv4): Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
  (conv5): Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
  (linear_ae): Linear(in_features=256, out_features=3, bias=True)
  (crf): ConditionalRandomField()
)


/home/xinke901/anaconda3/envs/de-cnn/lib/python3.7/site-packages/ipykernel_launcher.py:90: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


0/300
10/300
20/300
30/300
40/300
50/300
60/300
70/300
80/300
90/300
100/300
110/300
120/300
130/300
140/300
150/300
160/300
170/300
180/300
190/300
200/300
210/300
220/300
230/300
240/300
bert计算的loss是： 36.271186440677965
loss是： tensor(83.1781, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 31.52542372881356
loss是： tensor(98.1197, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 37.68115942028985
loss是： tensor(111.1966, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 34.14285714285714
loss是： tensor(99.3063, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 30.3125
loss是： tensor(98.4859, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 33.333333333333336
loss是： tensor(83.0194, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 37.84615384615385
loss是： tensor(109.2029, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 37.77777777777778
loss是： tensor(106.9251, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 33.333333333333336
loss是： tensor(8

bert计算的loss是： 44.39393939393939
loss是： tensor(113.6517, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 5.737704918032787
loss是： tensor(57.5522, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 44.34210526315789
loss是： tensor(120.6564, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 28.301886792452827
loss是： tensor(86.9473, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 31.61764705882353
loss是： tensor(82.0013, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 27.352941176470587
loss是： tensor(105.8284, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 11.35593220338983
loss是： tensor(68.5929, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 34.6875
loss是： tensor(55.8830, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 33.07692307692307
loss是： tensor(102.0371, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 38.771929824561404
loss是： tensor(114.3759, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 25.0
loss是： tensor(89.5088, 

bert计算的loss是： 38.10344827586207
loss是： tensor(96.1845, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 30.0
loss是： tensor(72.6549, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 29.420289855072465
loss是： tensor(111.7428, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 32.36363636363636
loss是： tensor(96.5018, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 25.074626865671643
loss是： tensor(104.8165, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 37.5
loss是： tensor(95.8905, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 32.666666666666664
loss是： tensor(76.1473, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 34.06779661016949
loss是： tensor(87.3670, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 43.52112676056338
loss是： tensor(93.2779, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 15.714285714285714
loss是： tensor(73.7798, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 14.571428571428571
loss是： tensor(57.3866, devi

...

bert计算的loss是： 32.27272727272727
loss是： tensor(108.9231, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 26.75324675324675
loss是： tensor(96.9188, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 36.12903225806451
loss是： tensor(86.0440, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 30.46153846153846
loss是： tensor(103.1172, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 32.59259259259259
loss是： tensor(82.8325, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 27.611940298507463
loss是： tensor(102.0593, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 18.985507246376812
loss是： tensor(78.5396, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 37.096774193548384
loss是： tensor(100.7866, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 32.0
loss是： tensor(79.6254, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 33.96551724137931
loss是： tensor(90.4429, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 64.84375
loss是： tensor(150.3075,

bert计算的loss是： 32.57575757575758
loss是： tensor(103.9164, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 20.434782608695656
loss是： tensor(86.1718, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 37.5
loss是： tensor(104.6207, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 21.475409836065573
loss是： tensor(67.5080, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 58.305084745762706
loss是： tensor(117.1223, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 34.516129032258064
loss是： tensor(90.5670, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 33.14814814814815
loss是： tensor(88.2178, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 21.311475409836067
loss是： tensor(65.0454, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 33.442622950819676
loss是： tensor(106.5871, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 28.4375
loss是： tensor(93.4784, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 44.516129032258064
loss是： tensor(100.57

bert计算的loss是： 27.89473684210526
loss是： tensor(72.7650, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 30.535714285714285
loss是： tensor(75.6266, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 59.26470588235294
loss是： tensor(153.1609, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 32.38095238095238
loss是： tensor(69.1493, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 28.676470588235293
loss是： tensor(71.9388, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 29.193548387096776
loss是： tensor(75.7451, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 41.25
loss是： tensor(97.4629, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 31.21212121212121
loss是： tensor(107.8355, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 30.923076923076923
loss是： tensor(87.3143, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 32.72727272727273
loss是： tensor(90.6492, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 38.208955223880594
loss是： tensor

bert计算的loss是： 30.806451612903224
loss是： tensor(82.2319, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 25.0
loss是： tensor(73.5348, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 26.73913043478261
loss是： tensor(75.0752, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 36.1764705882353
loss是： tensor(105.3326, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 31.285714285714285
loss是： tensor(117.2055, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 23.484848484848484
loss是： tensor(67.6938, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 34.92957746478873
loss是： tensor(118.2343, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 16.785714285714285
loss是： tensor(88.6550, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 27.666666666666664
loss是： tensor(76.6938, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 30.0
loss是： tensor(99.2759, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 31.71875
loss是： tensor(92.4071, device='cuda:

bert计算的loss是： 22.80701754385965
loss是： tensor(66.0438, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 34.30769230769231
loss是： tensor(90.6456, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 34.42622950819672
loss是： tensor(111.4881, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 38.80597014925373
loss是： tensor(89.9778, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 29.18918918918919
loss是： tensor(103.9123, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 33.93939393939394
loss是： tensor(92.3221, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 39.25373134328358
loss是： tensor(84.6911, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 24.75
loss是： tensor(44.6801, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 36.98412698412699
loss是： tensor(87.5853, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 43.89830508474576
loss是： tensor(104.0552, device='cuda:0', grad_fn=<AddBackward0>)
bert计算的loss是： 33.492063492063494
loss是： tensor(99